In [3]:
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 2.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 65.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 74.9 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
!pip install transformers[torch]
!pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
!pip install json

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch

model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/home/ubuntu/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
def tokenize_function(examples):
    return tokenizer(examples["Toxic Sentence"], examples["Explanation"], padding="max_length", truncation=True)

In [4]:
from datasets import load_dataset
dataset = load_dataset("json",data_files="gpt_2_train_data.json")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [5]:
# Step 6: Fine-Tune the Model
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].remove_columns(['Toxic Sentence', 'Explanation']),
    eval_dataset=tokenized_dataset["train"].remove_columns(['Toxic Sentence', 'Explanation']),
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    tokenizer=tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss


In [9]:
trainer.evaluate()

{'eval_loss': 3.8565833568573,
 'eval_runtime': 116.9516,
 'eval_samples_per_second': 1.702,
 'eval_steps_per_second': 0.111,
 'epoch': 1.0}

In [8]:
trainer.save_model("./toxic_reasoner_1")

In [82]:
def train_model(model_path, path):
    
    model_name = model_path
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    training_args = TrainingArguments(
        output_dir=f"./{path}",
        save_strategy="epoch",
        evaluation_strategy="epoch",
        num_train_epochs=1,
        learning_rate=0.5,
        per_device_train_batch_size=6,
        per_device_eval_batch_size=6,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="./logs"
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"].remove_columns(['Toxic Sentence', 'Explanation']),
        eval_dataset=tokenized_dataset["train"].remove_columns(['Toxic Sentence', 'Explanation']),
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
        tokenizer=tokenizer
    )

    trainer.train()
    
    return trainer
    

In [12]:
trainer1 = train_model("toxic_reasoner_1")

Epoch,Training Loss,Validation Loss
1,No log,3.841799
2,No log,3.797882


Checkpoint destination directory ./results/checkpoint-13 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [13]:
trainer1.save_model("./toxic_reasoner_2_3")

In [14]:
trainer1.evaluate()

{'eval_loss': 3.797881603240967,
 'eval_runtime': 126.8711,
 'eval_samples_per_second': 1.569,
 'eval_steps_per_second': 0.102,
 'epoch': 2.0}

In [ ]:
trainer2 = train_model("toxic_reasoner_2_3")

Epoch,Training Loss,Validation Loss
1,No log,3.793569


In [20]:
trainer2.save_model("./toxic_reasoner_4_5")

In [22]:
trainer2.evaluate()

{'eval_loss': 3.7733240127563477,
 'eval_runtime': 109.8123,
 'eval_samples_per_second': 1.812,
 'eval_steps_per_second': 0.082,
 'epoch': 2.0}

In [26]:
trainer3 = train_model("toxic_reasoner_4_5")

Epoch,Training Loss,Validation Loss
1,No log,5.703016
2,No log,4.872673


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [27]:
trainer3.save_model("./toxic_reasoner_6_7")

In [29]:
trainer4 = train_model("toxic_reasoner_6_7")

Epoch,Training Loss,Validation Loss
1,No log,4.181207
2,No log,3.688778


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [30]:
trainer4.save_model("./toxic_reasoner_8_9")

In [31]:
trainer5 = train_model("toxic_reasoner_8_9")

Epoch,Training Loss,Validation Loss
1,No log,3.512462
2,No log,3.278835


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [33]:
trainer5.save_model("./toxic_reasoner_10_11")

In [34]:
trainer6 = train_model("toxic_reasoner_10_11")

Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [36]:
trainer6.evaluate()

{'eval_loss': 2.5080654621124268,
 'eval_runtime': 113.9109,
 'eval_samples_per_second': 1.747,
 'eval_steps_per_second': 0.079,
 'epoch': 2.0}

In [37]:
trainer6.save_model("./toxic_reasoner_12_13")

In [39]:
trainer7 = train_model("toxic_reasoner_12_13")

Epoch,Training Loss,Validation Loss
1,No log,2.145200
2,No log,2.063081


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [41]:
trainer7.evaluate()

{'eval_loss': 2.0630810260772705,
 'eval_runtime': 112.0475,
 'eval_samples_per_second': 1.776,
 'eval_steps_per_second': 0.08,
 'epoch': 2.0}

In [43]:
trainer7.save_model("toxic_reasoner_14_15")

In [44]:
trainer8 = train_model("toxic_reasoner_14_15")

Epoch,Training Loss,Validation Loss
1,No log,1.896115
2,No log,1.799077


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [ ]:
trainer8.save_model("toxic_reasoner_16_17")

In [58]:
trainer9 = train_model("toxic_reasoner_16_17")

Epoch,Training Loss,Validation Loss
1,No log,1.350400
2,No log,1.699674


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [59]:
trainer9.save_model("toxic_reasoner_18_19")

In [60]:
trainer10 = train_model("toxic_reasoner_18_19")

Epoch,Training Loss,Validation Loss
1,No log,1.426986
2,No log,1.487062


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [63]:
trainer10.save_model("toxic_reasoner_20_21")

In [62]:
trainer11 = train_model("toxic_reasoner_20_21")

Epoch,Training Loss,Validation Loss
1,No log,1.174698
2,No log,1.483811


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [64]:
trainer11.save_model("toxic_reasoner_22_23")

In [65]:
trainer12 = train_model("toxic_reasoner_22_23")

Epoch,Training Loss,Validation Loss
1,No log,1.156881
2,No log,1.425956


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [70]:
trainer12.save_model("toxic_reasoner_24_25")

In [72]:
trainer13 = train_model("toxic_reasoner_24_25")

Epoch,Training Loss,Validation Loss
1,No log,1.022849
2,No log,1.338721


Checkpoint destination directory ./results1/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results1/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [73]:
trainer13.save_model("toxic_reasoner_26_27")

In [76]:
trainer14 = train_model("toxic_reasoner_26_27", "results2")

Epoch,Training Loss,Validation Loss
1,No log,1.133377
2,No log,1.257574


In [77]:
trainer14.save_model("toxic_reasoner_28_29")

In [78]:
trainer15 = train_model("toxic_reasoner_28_29", "results3")

Epoch,Training Loss,Validation Loss
1,No log,1.147412
2,No log,1.257591


In [79]:
trainer15.save_model("toxic_reasoner_30_31")

In [80]:
trainer16 = train_model("toxic_reasoner_30_31", "results4")

Epoch,Training Loss,Validation Loss
1,No log,1.104855
2,No log,1.433076


In [81]:
trainer16.save_model("toxic_reasoner_32_33")

In [83]:
trainer17 = train_model("toxic_reasoner_30_31", "results5")

Epoch,Training Loss,Validation Loss
1,No log,1.104855


In [84]:
trainer17.save_model("final_model")

In [85]:
from transformers import pipeline

In [86]:
qa = pipeline("text-generation", "final_model")

In [ ]:
qa("Are you a f*cking kidding me?", max_length=100)